<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# NLP Basics

**Deploying a Small DeepSeek Model Locally on Mac OS**

&copy; Dr. Yves J. Hilpisch

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

## Preparation

The following assumes usage of a Mac computer, running on Apple Silicon.

If you haven't installed it yet, insall `brew`:

    /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

Then, run:

    brew install cmake wget git

Install Miniconda if you haven't done so yet (from https://repo.anaconda.com/miniconda/):

    wget https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -O miniconda.sh
    bash miniconda.sh

Create a new environment:

    conda create -n deepseek python=3.10 -y
    conda activate deepseek

Install `PyTorch`:

    pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Install other required packages:

    pip install transformers accelerate sentencepiece safetensors
    conda install nodejs tqdm ipywidgets jupyterlab

## Testing `PyTorch`

In [ ]:
!git clone https://github.com/tpq-classes/natural_language_processing.git
import sys
sys.path.append('natural_language_processing')


In [ ]:
import torch
print(torch.backends.mps.is_available())  # Should return True
print(torch.backends.mps.is_built())  # Should return True

## Set the Model Path

Define where the (large) model files are getting stored:

    export HF_HOME=/path/to/custom/cache

Or with Python:

In [ ]:
import os
os.system('export HF_HOME=/Users/yves/Temp/deepseek')

## The Model(s)

There are a number of models from DeepSeek that can be deployed. Different types and different sizes They are all open source (open weights = model weights can be freely downloaded and used).

You find these models on the Hugging Face platform:

https://huggingface.co/models

Their first reasoning mode R1 is, for example, described in detail in this white paper:

https://arxiv.org/abs/2501.12948

See also the GitHub repository:

https://github.com/deepseek-ai/DeepSeek-R1

## Running the Model

In [ ]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

# Check if MPS (Apple Metal) is available; fallback to CPU if not
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Load model and tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# Move model to the selected device
model.to(device)

In [ ]:
# Input prompt
input_text = "What is the future of AI?"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Measure inference time
start_time = time.time()

# Generate text
with torch.no_grad():
    output = model.generate(**inputs, max_length=1250)

end_time = time.time()

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Calculate performance metrics
num_tokens = output.shape[1]  # Number of tokens in the output
elapsed_time = end_time - start_time  # Time taken in seconds
tokens_per_second = num_tokens / elapsed_time if elapsed_time > 0 else float("inf")

# Print results
print("\nGenerated Response:")
print(generated_text)
print("\nPerformance Metrics:")
print(f"Generated {num_tokens} tokens in {elapsed_time:.2f} seconds")
print(f"Tokens per second: {tokens_per_second:.2f}")

## Alternative Code

In [ ]:
from tqdm import tqdm

In [ ]:
# Test input
prompt = "Explain the significance of Python in quantitative finance."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Warm-up (to stabilize performance measurement)
for _ in range(3):
    _ = model.generate(**inputs, max_new_tokens=10)

# Measure inference time
num_trials = 5
total_tokens = 0
total_time = 0

for _ in tqdm(range(num_trials), desc="Running Inference Tests"):
    start_time = time.time()
    output = model.generate(**inputs, max_new_tokens=250)
    end_time = time.time()

    num_tokens = output.shape[-1]
    total_tokens += num_tokens
    total_time += (end_time - start_time)

    # Print output for verification
    print(tokenizer.decode(output[0], skip_special_tokens=True))

# Calculate tokens per second
tokens_per_second = total_tokens / total_time
print(f"\nAverage Tokens per Second: {tokens_per_second:.2f}")

In [ ]:
# Input prompt
input_text = "What is the future of AI?"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Measure inference time
start_time = time.time()

# Generate text
with torch.no_grad():
    output = model.generate(**inputs, max_length=2000)
    
    end_time = time.time()
    
    # Decode the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Calculate performance metrics
    num_tokens = output.shape[1]  # Number of tokens in the output
    elapsed_time = end_time - start_time  # Time taken in seconds
    tokens_per_second = num_tokens / elapsed_time if elapsed_time > 0 else float("inf")
    
    # Print results
    print("\nGenerated Response:")
    print(generated_text)
    print("\nPerformance Metrics:")
    print(f"Generated {num_tokens} tokens in {elapsed_time:.2f} seconds")
    print(f"Tokens per second: {tokens_per_second:.2f}")

## Example Output

Generated Response:

What is the future of AI? What are the potential impacts of AI on society?
The AI is supposed to be a powerful tool for the future of society, but I feel that there are still some challenges in the field. Maybe I should look into the future of AI from a different angle, perhaps focusing on ethical and societal implications rather than just technical aspects.

Looking at the potential impacts of AI on society, it seems like AI could have a transformative impact on various areas like healthcare, education, and economy. But I'm not sure how significant these impacts would be. I mean, healthcare is already getting a lot of attention, but what about the other areas? Also, I'm concerned about the ethical considerations. How does AI make decisions, and can it lead to biases or unfairness?

Another thing I'm thinking about is the role of AI in job markets. Maybe AI can change the way people get employed by automating certain tasks, but I'm not sure how that affects the job market itself. There might be a lot of job displacement, but I also don't know if AI would make the job creation process more efficient or change the nature of jobs.

I also wonder about the potential for AI to be used in areas like criminal justice, where it could help with surveillance or detection. But I'm not sure if that would be a positive thing or just another tool in the system. Plus, there are concerns about privacy and data security when AI is used in these areas.

Overall, I feel like there's a lot more to consider about the future of AI and its societal impacts. Maybe I should look into case studies or expert opinions to get a better understanding of how AI is currently being used and the potential issues that could arise.

One more thing I'm thinking about is the balance between innovation and regulation. As AI becomes more powerful, there might come a need for more stringent regulations to ensure that AI doesn't cause harm. But I'm not sure how to go about creating such regulations or how existing regulations can be updated and improved.

I also recall that AI is still a relatively new technology, so there might be a lot of uncertainty about its capabilities and limitations. This uncertainty could make it difficult for society to fully harness AI's potential, especially in areas where access to AI tools is limited.

In summary, I need to research more about AI's future, focusing on ethical, societal, and practical aspects. I also need to understand the current state of AI technology, including its applications and challenges. Additionally, I should look into case studies, expert opinions, and research to get a well-rounded perspective on the potential impacts of AI on society.

Now, I'm feeling a bit overwhelmed with all these ideas. I need to organize my thoughts and start gathering information from reliable sources. Maybe I can look into academic papers, industry reports, and government documents on AI. Also, watching news coverage of AI-related events could provide insights into the current state and future projections.

Another approach could be to identify key areas where AI is making significant impacts and explore how it could influence those areas. For example, in healthcare, AI is used for diagnostics and treatment plans, which could lead to better patient outcomes. In education, AI is used for personalized learning experiences, which could improve student engagement and academic performance.

I also think about the role of education in shaping future AI developers and researchers. If education is more accessible, there might be more interest in pursuing AI-related careers. Conversely, if there's a lack of awareness about AI's potential, it could limit the number of professionals in this field.

Furthermore, I consider the societal implications of AI, such as its role in social governance and policy-making. AI can help in analyzing large datasets to inform decisions, which could lead to more informed and efficient governance. However, I also need to be cautious about how AI might be used to manipulate or control data, which could raise ethical concerns.

I also wonder about the potential for AI to disrupt traditional industries. For instance, in manufacturing, AI can optimize production processes, leading to cost savings and increased efficiency. In logistics, AI can help in route optimization and delivery scheduling, which could improve supply chain management.

Moreover, I think about the role of AI in addressing global challenges such as climate change and pandemics. AI can provide tools for climate modeling and pandemic response, which could help in making informed decisions and mitigating their impacts. However, I also need to consider the ethical implications of these applications, such as their impact on vulnerable populations or their ability to lead to unintended consequences.

Another aspect I'm considering is the ethical use of AI. As AI becomes more integrated into daily life, there's a need to establish guidelines and ethical frameworks to ensure that AI is used responsibly. This could involve regulations for data privacy, job standards, and decision-making processes.

I also think about the potential for AI to enhance creativity and problem-solving skills. AI can generate ideas and solve complex problems, which could have a positive impact on education and society. However, I also need to be cautious about the potential biases and limitations of AI in generating ideas and solving problems.

In addition, I consider the potential for AI to improve international collaboration and cooperation. AI can bridge communication gaps and facilitate knowledge sharing between countries, which could lead to better global governance and problem-solving.

Overall, I need to organize my thoughts around these various aspects and seek out the information and insights that can help me understand the future of AI and its societal impacts. By doing so, I can make informed decisions and contribute to the development of a more ethical and effective AI future.
</think>

The future of AI is poised to revolutionize society in profound ways, with significant implications across various sectors. Here is an organized summary of the potential impacts and considerations:

### Future of AI
1. **Technological Advancements**: AI is expected to become increasingly sophisticated, capable of handling complex tasks such as creativity, decision-making, and problem-solving. This could lead to transformative changes in industries like healthcare, education, and economy.

2. **Ethical and Societal Impacts**: AI's role is likely to be both a double-edged sword and catalyst for change. Key considerations include:
   - **Ethical Considerations**: Addressing bias, fairness, and transparency in AI algorithms to ensure equitable outcomes.
   - **Societal Impacts**: Assessing the well-being of individuals affected by AI, including those who may be disproportionately impacted by automation.

3. **Future Directions**: Focusing on ethical and societal dimensions, AI will likely influence fields such as healthcare, education, and criminal justice, potentially changing how people work and interact.

### Potential Societal Impacts
1. **Healthcare**: Transformative potential, including personalized medicine and disease detection. However, the impact on underserved populations must be carefully considered.
2. **Education**: Personalized learning and efficient resource allocation. Challenges include ensuring equitable access and addressing potential biases.
3. **Economy**: Automation and AI could lead to job displacement and structural changes. The role of AI in innovation and economic growth must be balanced against its negative effects.
4. **Criminal Justice**: Potential role in surveillance and crime detection, though concerns about privacy and bias must be addressed.
5. **Job Markets**: Automation could change job structures, leading to job displacement but also new roles in AI development and innovation.

### Case Studies and Expert Opinions
- Reviewing academic papers, industry reports, and government documents on AI provides insights into current applications and challenges.
- Watching news coverage of AI-related events offers real-world context and projections.

### Key Considerations
1. **Regulation and Innovation**: Striking a balance between innovation and regulation to ensure AI's responsible use. This includes developing regulations for data privacy and job standards.
2. **Education and Career Development**: Enhancing accessibility to AI-related fields to attract and retain talent.
3. **Societal Governance**: Utilizing AI for better data analysis and informed governance, while addressing ethical concerns.
4. **International Collaboration**: Bridging communication gaps and fostering knowledge sharing to enhance global problem-solving.

### Conclusion
The future of AI is a complex interplay of technological innovation and societal impact. By focusing on ethical, societal, and practical aspects, society can better harness AI's potential while mitigating its risks.

Performance Metrics:
Generated 1678 tokens in 123.93 seconds
Tokens per second: 13.54

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>